In [1]:
import numpy as np 
import string
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/darkness/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/darkness/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre Processing

### Text Retrival

In [2]:
f = open("Text.txt")
data = f.readlines()[6:]
f.close()

In [3]:
data

['Writer-director Emerald Fennell’s Saltburn is that rare film that has the internet in a chokehold while simultaneously digging up references to Shakespeare, Evelyn Waugh and Greek mythology. It’s been a while since I’ve watched a film that centers on a group of friends at Oxford and turns into a country house thriller. Set in the mid 2000s, Saltburn, follows a lonely and outcast first year student Oliver Quick (Barry Keoghan) as he is drawn into the inner circle of the wealthy and beautiful Felix Catton (Jacob Elordi). While comparisons to other novels and films abound (some directly referenced and others not) and certain scenes have simultaneously revolted and intrigued viewers, unsurprisingly, I find myself thinking about the symbolism from Greek mythology.\n',
 'Major spoilers for Saltburn follow.\n',
 'The most obvious symbolism in Saltburn is found in the figure of the Minotaur which stands menacingly in the estate’s maze and is the scene of Felix’s murder. In Greek mythology, t

### Tokenization

In [4]:
# Sentence Splitting
sentences = []
for i in data:
    sentences += nltk.sent_tokenize(i)
# Word Splitting
words_in_sentence = []
for i in sentences:

    words_in_sentence.append(nltk.word_tokenize(i))

In [5]:
print(words_in_sentence)

[['Writer-director', 'Emerald', 'Fennell', '’', 's', 'Saltburn', 'is', 'that', 'rare', 'film', 'that', 'has', 'the', 'internet', 'in', 'a', 'chokehold', 'while', 'simultaneously', 'digging', 'up', 'references', 'to', 'Shakespeare', ',', 'Evelyn', 'Waugh', 'and', 'Greek', 'mythology', '.'], ['It', '’', 's', 'been', 'a', 'while', 'since', 'I', '’', 've', 'watched', 'a', 'film', 'that', 'centers', 'on', 'a', 'group', 'of', 'friends', 'at', 'Oxford', 'and', 'turns', 'into', 'a', 'country', 'house', 'thriller', '.'], ['Set', 'in', 'the', 'mid', '2000s', ',', 'Saltburn', ',', 'follows', 'a', 'lonely', 'and', 'outcast', 'first', 'year', 'student', 'Oliver', 'Quick', '(', 'Barry', 'Keoghan', ')', 'as', 'he', 'is', 'drawn', 'into', 'the', 'inner', 'circle', 'of', 'the', 'wealthy', 'and', 'beautiful', 'Felix', 'Catton', '(', 'Jacob', 'Elordi', ')', '.'], ['While', 'comparisons', 'to', 'other', 'novels', 'and', 'films', 'abound', '(', 'some', 'directly', 'referenced', 'and', 'others', 'not', ')',

In [6]:
apostrophy = words_in_sentence[0][3] + '”“‘—'

In [7]:
words_in_sentence = [[i for i in j if i not in ( string.punctuation + apostrophy ) ] for j in words_in_sentence]
words_in_sentence = [[ i for i in j] for j in words_in_sentence]

In [8]:
print(words_in_sentence)

[['Writer-director', 'Emerald', 'Fennell', 's', 'Saltburn', 'is', 'that', 'rare', 'film', 'that', 'has', 'the', 'internet', 'in', 'a', 'chokehold', 'while', 'simultaneously', 'digging', 'up', 'references', 'to', 'Shakespeare', 'Evelyn', 'Waugh', 'and', 'Greek', 'mythology'], ['It', 's', 'been', 'a', 'while', 'since', 'I', 've', 'watched', 'a', 'film', 'that', 'centers', 'on', 'a', 'group', 'of', 'friends', 'at', 'Oxford', 'and', 'turns', 'into', 'a', 'country', 'house', 'thriller'], ['Set', 'in', 'the', 'mid', '2000s', 'Saltburn', 'follows', 'a', 'lonely', 'and', 'outcast', 'first', 'year', 'student', 'Oliver', 'Quick', 'Barry', 'Keoghan', 'as', 'he', 'is', 'drawn', 'into', 'the', 'inner', 'circle', 'of', 'the', 'wealthy', 'and', 'beautiful', 'Felix', 'Catton', 'Jacob', 'Elordi'], ['While', 'comparisons', 'to', 'other', 'novels', 'and', 'films', 'abound', 'some', 'directly', 'referenced', 'and', 'others', 'not', 'and', 'certain', 'scenes', 'have', 'simultaneously', 'revolted', 'and', '

In [9]:
Stop_word = stopwords.words('english') + list(string.ascii_lowercase) + list(string.ascii_uppercase)

words_nostop_in_sentence = []
for i in words_in_sentence:
    temp = [word for word in i if word not in Stop_word]
    words_nostop_in_sentence.append(temp)
print(words_nostop_in_sentence)

[['Writer-director', 'Emerald', 'Fennell', 'Saltburn', 'rare', 'film', 'internet', 'chokehold', 'simultaneously', 'digging', 'references', 'Shakespeare', 'Evelyn', 'Waugh', 'Greek', 'mythology'], ['It', 'since', 'watched', 'film', 'centers', 'group', 'friends', 'Oxford', 'turns', 'country', 'house', 'thriller'], ['Set', 'mid', '2000s', 'Saltburn', 'follows', 'lonely', 'outcast', 'first', 'year', 'student', 'Oliver', 'Quick', 'Barry', 'Keoghan', 'drawn', 'inner', 'circle', 'wealthy', 'beautiful', 'Felix', 'Catton', 'Jacob', 'Elordi'], ['While', 'comparisons', 'novels', 'films', 'abound', 'directly', 'referenced', 'others', 'certain', 'scenes', 'simultaneously', 'revolted', 'intrigued', 'viewers', 'unsurprisingly', 'find', 'thinking', 'symbolism', 'Greek', 'mythology'], ['Major', 'spoilers', 'Saltburn', 'follow'], ['The', 'obvious', 'symbolism', 'Saltburn', 'found', 'figure', 'Minotaur', 'stands', 'menacingly', 'estate', 'maze', 'scene', 'Felix', 'murder'], ['In', 'Greek', 'mythology', '

### POS Tagging

In [10]:
POS = []
for i in words_nostop_in_sentence:
    POS.append(nltk.pos_tag(i))

print(POS)

[[('Writer-director', 'NNP'), ('Emerald', 'NNP'), ('Fennell', 'NNP'), ('Saltburn', 'NNP'), ('rare', 'JJ'), ('film', 'NN'), ('internet', 'NN'), ('chokehold', 'VBD'), ('simultaneously', 'RB'), ('digging', 'VBG'), ('references', 'NNS'), ('Shakespeare', 'NNP'), ('Evelyn', 'NNP'), ('Waugh', 'NNP'), ('Greek', 'NNP'), ('mythology', 'NN')], [('It', 'PRP'), ('since', 'IN'), ('watched', 'VBN'), ('film', 'NN'), ('centers', 'NNS'), ('group', 'NN'), ('friends', 'VBZ'), ('Oxford', 'NNP'), ('turns', 'VBZ'), ('country', 'NN'), ('house', 'NN'), ('thriller', 'NN')], [('Set', 'NNP'), ('mid', 'VBD'), ('2000s', 'CD'), ('Saltburn', 'NNP'), ('follows', 'VBZ'), ('lonely', 'RB'), ('outcast', 'VBN'), ('first', 'JJ'), ('year', 'NN'), ('student', 'NN'), ('Oliver', 'NNP'), ('Quick', 'NNP'), ('Barry', 'NNP'), ('Keoghan', 'NNP'), ('drawn', 'NN'), ('inner', 'JJ'), ('circle', 'NN'), ('wealthy', 'JJ'), ('beautiful', 'JJ'), ('Felix', 'NNP'), ('Catton', 'NNP'), ('Jacob', 'NNP'), ('Elordi', 'NNP')], [('While', 'IN'), ('co

### Lemmatization

In [11]:
root_per_sentence = []
obj = nltk.WordNetLemmatizer()

for i in words_nostop_in_sentence:
    temp  = [obj.lemmatize(j) for j in i ] 
    root_per_sentence.append(temp)
print(root_per_sentence)

[['Writer-director', 'Emerald', 'Fennell', 'Saltburn', 'rare', 'film', 'internet', 'chokehold', 'simultaneously', 'digging', 'reference', 'Shakespeare', 'Evelyn', 'Waugh', 'Greek', 'mythology'], ['It', 'since', 'watched', 'film', 'center', 'group', 'friend', 'Oxford', 'turn', 'country', 'house', 'thriller'], ['Set', 'mid', '2000s', 'Saltburn', 'follows', 'lonely', 'outcast', 'first', 'year', 'student', 'Oliver', 'Quick', 'Barry', 'Keoghan', 'drawn', 'inner', 'circle', 'wealthy', 'beautiful', 'Felix', 'Catton', 'Jacob', 'Elordi'], ['While', 'comparison', 'novel', 'film', 'abound', 'directly', 'referenced', 'others', 'certain', 'scene', 'simultaneously', 'revolted', 'intrigued', 'viewer', 'unsurprisingly', 'find', 'thinking', 'symbolism', 'Greek', 'mythology'], ['Major', 'spoiler', 'Saltburn', 'follow'], ['The', 'obvious', 'symbolism', 'Saltburn', 'found', 'figure', 'Minotaur', 'stand', 'menacingly', 'estate', 'maze', 'scene', 'Felix', 'murder'], ['In', 'Greek', 'mythology', 'Minotaur', 

## TF -IDF VECTOR REPRESENTATION

In [12]:
unique_words = []
for i in root_per_sentence:
    for j in i :
        if j not in unique_words:
            unique_words.append(j)
unique_words = sorted(unique_words)

In [13]:
tf = []
idf = [0 for _ in range(len(unique_words))]
for i in range(len(root_per_sentence)):
    temp = [0 for _ in range(len(unique_words))]
    for j in root_per_sentence[i]:
        temp[unique_words.index(j)] = 1 / len(root_per_sentence[i])
        idf[unique_words.index(j)] += 1
    tf.append(temp) 

tf = np.array(tf)
idf = np.array(idf)
idf = np.array(np.log2(len(root_per_sentence))) - np.log2(idf)
idf = idf.reshape(1,-1)

In [14]:
# TF - IDF Vectorization
import pandas as pd
Data_TF_IDF = pd.DataFrame(index=["sentence-" + str(i+1) for i in range(len(root_per_sentence) )],columns=sorted(unique_words),data=tf)

In [15]:
vector_sentence1 = (np.array(Data_TF_IDF.loc['sentence-1']).reshape(1,-1) * idf)
vector_sentence2 = (np.array(Data_TF_IDF.loc['sentence-2']).reshape(1,-1) * idf)

In [16]:
def mag(vct):
    return np.sum(vct**2)

In [17]:
Sim_tf_idf = np.dot(vector_sentence1,vector_sentence2.T) / (mag(vector_sentence1) * mag(vector_sentence2))

### Similarity score(sent1,sent2) is 0.010788134718958942

## Bag of Words

In [18]:
tf_bow = []
for i in range(len(root_per_sentence)):
    temp = [0 for _ in range(len(unique_words))]
    for j in root_per_sentence[i]:
        temp[unique_words.index(j)] = 1 
    tf_bow.append(temp) 
tf_bow = np.array(tf_bow)

In [19]:
Data_bow = pd.DataFrame(index=["sentence-" + str(i+1) for i in range(len(root_per_sentence) )],columns=sorted(unique_words),data=tf_bow)

In [20]:
vector_sentence1_bow = (np.array(Data_bow.loc['sentence-1']).reshape(1,-1) )
vector_sentence2_bow = (np.array(Data_bow.loc['sentence-2']).reshape(1,-1) )
Sim_bow =  np.dot(vector_sentence1_bow,vector_sentence2_bow.T) / (mag(vector_sentence1_bow) * mag(vector_sentence2_bow))

### Similarity score(sent1,sent2)  0.005208333333333333

## WORD2VEC(CBOW)

In [43]:
from gensim.models import Word2Vec

model = Word2Vec(root_per_sentence,min_count=1,vector_size=150,window=5,workers=5,epochs=100,sg=1)

In [48]:
Vector_sentence = [model.wv[root_per_sentence[i]].sum(axis=0).reshape(1,150) for i in range(len(root_per_sentence))]

In [49]:
Sim_word2vec = np.dot(Vector_sentence[0],Vector_sentence[1].T) / ((mag(Vector_sentence[0])) * mag(Vector_sentence[1]))

### Similarity score(sent1,sent2) is 0.0013258324

## END